In [1]:
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras import backend as K
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from patchify import patchify, unpatchify
from tensorflow.keras import backend as K
from skimage.measure import label

def f1(y_true, y_pred, threshold=0.3):
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    TP = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    Positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    Pred_Positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = TP / (Pred_Positives + tf.keras.backend.epsilon())
    recall = TP / (Positives + tf.keras.backend.epsilon())
    return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
    
@tf.keras.utils.register_keras_serializable()
def f1_metric(y_true, y_pred):
    return f1(y_true, y_pred, threshold=0.3)


def weighted_binary_crossentropy(y_true, y_pred):
    """
    Weighted binary cross-entropy to address class imbalance.
    """
    # Define weights for foreground (root, shoot, seed) and background
    weight_foreground = 10.0
    weight_background = 1.0

    # Compute weighted binary cross-entropy
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    weights = tf.where(y_true == 1, weight_foreground, weight_background)
    loss = K.binary_crossentropy(y_true, y_pred)
    weighted_loss = loss * weights
    return K.mean(weighted_loss)
def dice_loss(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2. * intersection + 1e-7) / (union + 1e-7)
    return 1 - dice

@tf.keras.utils.register_keras_serializable()
def combined_loss(y_true, y_pred):
    return 0.5 * dice_loss(y_true, y_pred) + 0.5 * weighted_binary_crossentropy(y_true, y_pred)





2025-01-09 09:07:30.301581: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-09 09:07:30.301623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-09 09:07:30.303069: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-09 09:07:30.310664: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
import logging

# Logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("pipeline_log.txt"), logging.StreamHandler()],
)

def preprocess_image(image_path):
    """Load and preprocess the input image."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")
    return image

def extract_petri_dish(image):
    """Extract the largest contour assumed to be the Petri dish."""
    _, thresholded = cv2.threshold(image, 57, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        logging.warning("No contours detected for Petri dish.")
        return image, np.ones_like(image)

    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    cropped_image = image[y:y+h, x:x+w]
    return cropped_image

def predict_root_mask(image, model, patch_size=128, stride=64):
    """Predict root mask using a patch-based model."""
    h, w = image.shape
    patches = []
    positions = []
    for y in range(0, h - patch_size + 1, stride):
        for x in range(0, w - patch_size + 1, stride):
            patch = image[y:y+patch_size, x:x+patch_size]
            patch_rgb = np.stack([patch]*3, axis=-1)
            patches.append(patch_rgb)
            positions.append((y, x))

    patches = np.array(patches) / 255.0
    predictions = model.predict(patches, verbose=0)

    reconstructed = np.zeros((h, w), dtype=np.float32)
    counts = np.zeros((h, w), dtype=np.float32)
    for pred, (y, x) in zip(predictions, positions):
        pred = pred[..., 0]
        reconstructed[y:y+patch_size, x:x+patch_size] += pred
        counts[y:y+patch_size, x:x+patch_size] += 1
    return (reconstructed / np.maximum(counts, 1) > 0.3).astype(np.uint8)

def connect_roots(mask):
    """Dilate the mask to connect fragmented roots."""
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
    return cv2.dilate(mask, kernel, iterations=27)

def filter_and_select_largest_objects(mask, min_area=500, max_objects=5):
    """Filter and retain the largest root components."""
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    valid_objects = [(i, stats[i, cv2.CC_STAT_AREA]) for i in range(1, num_labels) if stats[i, cv2.CC_STAT_AREA] >= min_area]
    largest_objects = sorted(valid_objects, key=lambda x: x[1], reverse=True)[:max_objects]

    filtered_mask = np.zeros_like(mask, dtype=np.uint8)
    for obj_id, _ in largest_objects:
        filtered_mask[labels == obj_id] = 255
    return filtered_mask, largest_objects

def measure_bounding_boxes(filtered_mask):
    """Measure roots with bounding boxes."""
    contours, _ = cv2.findContours(filtered_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = [cv2.boundingRect(contour) for contour in contours]
    bounding_boxes = sorted(bounding_boxes, key=lambda b: b[0])
    return bounding_boxes

def find_lowest_point(mask, bounding_box):
    """Find the lowest point in a bounding box."""
    x, y, w, h = bounding_box
    roi = mask[y:y+h, x:x+w]
    coordinates = np.column_stack(np.where(roi > 0))
    if len(coordinates) == 0:
        return None
    lowest_local = coordinates[np.argmax(coordinates[:, 0])]
    return lowest_local + [y, x]

def convert_to_mm(pixel_coords, image_shape, plate_size_mm=150):
    """Convert pixel coordinates to mm-space."""
    h, w = image_shape
    conversion_factor_x = plate_size_mm / w
    conversion_factor_y = plate_size_mm / h
    return np.array([pixel_coords[1] * conversion_factor_x, pixel_coords[0] * conversion_factor_y])

# Directory containing images
input_dir = "kaggle_test"
model_path = "232430_unet_model_128px_v7md.keras"
plate_position_robot = np.array([0.10775, 0.088 - 0.026, 0.057])  # Adjust based on your setup

# Load the model
model = tf.keras.models.load_model(model_path)

# Process all images in the directory
all_root_tip_coords = {}

for image_name in sorted(os.listdir(input_dir)):
    if not image_name.endswith((".png", ".jpg", ".jpeg")):
        continue

    image_path = os.path.join(input_dir, image_name)
    logging.info(f"Processing image: {image_name}")

    try:
        # Process the image and extract root tip coordinates
        image = preprocess_image(image_path)
        petri_dish = extract_petri_dish(image)
        predicted_mask = predict_root_mask(petri_dish, model)
        connected_mask = connect_roots(predicted_mask)
        filtered_mask, _ = filter_and_select_largest_objects(connected_mask, min_area=500, max_objects=5)
        bounding_boxes = measure_bounding_boxes(filtered_mask)

        root_tip_coords = []
        for box in bounding_boxes:
            point = find_lowest_point(filtered_mask, box)
            if point is not None:
                mm_coords = convert_to_mm(point, petri_dish.shape)
                robot_coords = np.append(mm_coords, 0) + plate_position_robot
                root_tip_coords.append(robot_coords)

        all_root_tip_coords[image_name] = root_tip_coords
        logging.info(f"Extracted Root Tip Coordinates for {image_name}: {root_tip_coords}")

    except Exception as e:
        logging.error(f"Error processing image {image_name}: {e}")

# Expose all_root_tip_coords for import
def get_root_tip_coords():
    return all_root_tip_coords

# Log all extracted root tip coordinates
logging.info("All Extracted Root Tip Coordinates:")
for img_name, coords in all_root_tip_coords.items():
    logging.info(f"{img_name}: {coords}")


2025-01-09 09:07:33.620142: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2025-01-09 09:07:33.620528: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 25223 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:25:00.0, compute capability: 8.6
INFO: Processing image: test_image_1.png
2025-01-09 09:07:39.390672: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
INFO: Extracted Root Tip Coordinates for test_image_1.png: [array([ 6.89359458, 43.1754469 ,  0.057     ]), array([1.65251159e+01, 6.26390207e+01, 5.70000000e-02]), array([23.2015114 , 54.59262704,  0.057     ]), array([7.61201542e+01, 3.94784552e+01, 5.70000000e-02]), array([1.25864773e+02, 4.50239427e+01, 5.70000000e-02])]
INFO: Processing image: test_image_10.png
I

In [5]:
from sim_class import Simulation
import os
from pid_class import PIDController


# Get the directory of the current script
script_dir = os.path.dirname(os.path.abspath(__file__))

# Change the current working directory to the script's directory
os.chdir(script_dir)

print("Current working directory:", os.getcwd())

# Initialize simulation with one agent
num_agents = 1
simulation = Simulation(num_agents=num_agents, render=True)

# Define custom PID parameters for each robot (optional)
custom_pid_params = [
    {'x': {'Kp': 2.0, 'Ki': 0.1, 'Kd': 0.05}, 'y': {'Kp': 2.0, 'Ki': 0.1, 'Kd': 0.05}, 'z': {'Kp': 3.0, 'Ki': 0.2, 'Kd': 0.1}}
]

# Set PID parameters (optional, otherwise uses defaults in Simulation)
simulation.set_pid_params(custom_pid_params)

# Define the target positions for each robot
setpoints = all_root_tip_coords
simulation.set_pid_setpoints(setpoints)

# Run the simulation loop
try:
    num_steps = 2400  # Number of simulation steps
    for step in range(num_steps):
        simulation.control_pid()  # Update velocities based on PID control
        p.stepSimulation()  # Step the simulation

        # Optional: Slow down the simulation for better visualization
        time.sleep(1.0 / 240.0)

        # Optional: Print pipette positions for debugging
        if step % 100 == 0:
            states = simulation.get_states()
            print(f"Step {step}: {states}")

except KeyboardInterrupt:
    print("Simulation interrupted.")

finally:
    simulation.close()


ModuleNotFoundError: No module named 'pid_controller'